In [1]:
# Setup when required
#!rm -rf /home/jupyter/.local/lib/python2.7/site-packages/sciunit* ./sciunit
#!git clone -b test_m2m https://github.com/appukuttan-shailesh/sciunit
#!pip install -q ./sciunit

In [2]:
import sciunit

### Creating Dummy Capability

In [3]:
class HasValue(sciunit.Capability):
    """Indicates that the model has an attribute named 'value'"""

    def get_value(self):
        """Must return a float value."""
        raise NotImplementedError()

### Creating Dummy Model 1

In [4]:
class model_v1(sciunit.Model, HasValue):     
        def __init__(self, name="Model_v1"):
            self.set_value()
            sciunit.Model.__init__(self, name=name)
            self.description = "Dummy model 1 for testing Test_M2M"

        def set_value(self):
            self.value = {"value" : "260 um"}

        def get_value(self):
            return self.value


### Creating Dummy Model 2
(could simply have created another instance of above model with different values)

In [5]:
class model_v2(sciunit.Model, HasValue):     
        def __init__(self, name="Model_v2"):
            self.set_value()
            sciunit.Model.__init__(self, name=name)
            self.description = "Dummy model 2 for testing Test_M2M"

        def set_value(self):
            self.value = {"value" : "250 um"}

        def get_value(self):
            return self.value

### Creating Model to Model Test

In [6]:
import sciunit
import sciunit.scores
import quantities

class CellValueTest_M2M(sciunit.TestM2M):
    """Dummy Test"""
    score_type = sciunit.scores.FloatScore
    units = quantities.um
    description = ("Tests the parameter 'value' between two models")

    def __init__(self, observation=None, name="DummyTest-M2M"):
        sciunit.TestM2M.__init__(self,observation,name)
        self.required_capabilities += (HasValue,)        

    def format_data(self, data):
        """
        This accepts data input in the form:
        ***** (prediction) *****
        {"value": "x um"}

        It splits the value to numeric quantities
        and their units (via quantities package).
        """
        for key,val in data.items():
            try:
                quantity_parts = val.split(" ")
                number = float(quantity_parts[0])
                units_str = " ".join(quantity_parts[1:])
                assert (units_str == self.units.symbol)
                data[key] = quantities.Quantity(number, self.units)
            except ValueError:
                raise sciunit.Error("Values not in appropriate format. Required units: um")
        return data
    
    def validate_observation(self, observation):
        if self.observation:
            self.observation = self.format_data(observation)

    def generate_prediction(self, model, verbose=False):
        """Implementation of sciunit.Test.generate_prediction."""
        self.model_name = model.name
        prediction = model.get_value()
        prediction = self.format_data(prediction)
        return prediction

    def compute_score(self, prediction1, prediction2, verbose=False):
        """Implementation of sciunit.Test.score_prediction."""
        #print "prediction 1 = ", prediction1
        #print "prediction 2 = ", prediction2
        score = sciunit.scores.FloatScore(prediction1["value"].item() - prediction2["value"].item())
        score.description = "Difference between model predictions"
        return score

In [7]:
#valueTest = CellValueTest_M2M()
valueTest = CellValueTest_M2M(observation={"value": "275 um"})
Model_v1 = model_v1()
Model_v2 = model_v2()
sm = valueTest.judge([Model_v1, Model_v2])

In [8]:
print sm

            observation Model_v1 Model_v2
observation           0       15       25
Model_v1            -15        0       10
Model_v2            -25      -10        0


In [9]:
print sm.index
print sm.columns

Index([observation, Model_v1, Model_v2], dtype='object')
Index([observation, Model_v1, Model_v2], dtype='object')


In [10]:
print sm['Model_v1']
print "-----"
print sm[Model_v1]

observation    -15
Model_v1         0
Model_v2        10
Name: Model_v1, dtype: object
-----
observation    -15
Model_v1         0
Model_v2        10
Name: Model_v1, dtype: object


In [11]:
print sm['Model_v1']['Model_v1']
print "-----"
print sm[Model_v1][Model_v1]

0
-----
0


In [12]:
print sm['Model_v1']['Model_v2']
print "-----"
print sm[Model_v1][Model_v2]

10
-----
10


In [13]:
print sm.iloc[0][1]
print sm.loc[Model_v1][Model_v2]
print sm.iat[0,1]
print sm.at[Model_v1,Model_v2]
print sm.get_value(Model_v1, Model_v2)

15
10
15
10
10


In [14]:
print sm[valueTest][Model_v1]
print "-----"
print sm["observation"][Model_v1]

15
-----
15
